# "It's about time to take your medication!"
## or how to write a friendly reminder bot ;-)

* Florian Wilhelm, Blue Yonder
* EuroPython 2015, Bilbao, Spain
* [https://github.com/blue-yonder/medbot/](https://github.com/blue-yonder/medbot/)

# Outline
* Motivation
* XMPP/Hangups
* Event-Driven/Asynchronous Programming
* OAUTH2
* Credits

# Motivation

Why would anyone write a chat bot?

# Motivation

Learn about...
* the basics of chat protocols like XMPP
* the concepts of event-driven/asynchronous programming
* how to write a small Google App
* help a friend with diabetes


# XMPP
* Extensible Messaging and Presence Protocol based on XML
* developed by the Jabber open-source community in 1999 for near real-time instant messaging
* GTalk is based on XMPP but Hangout switched to a proprietary protocol
* Facebook provided an XMPP API until May, 2015
* AIM limited XMPP support

# SleekXMPP
* MIT licensed XMPP library
* Python 2.6/3.1+
* Featured in the O'Reilly book [XMPP: The Definitive Guide](http://shop.oreilly.com/product/9780596521271.do) by Kevin Smith, Remko Tronçon, and Peter Saint-Andre
* **Design goals:** low number of dependencies, every XEP as a plugin and rewarding to work with

# Google Hangouts protocol
* replaces Google Talk, Google+ Messenger and the Google+ video chat system
* proprietary protocol, cannot be integrated with multi-chat clients like Pidgin or Adium
* Some more features like photo messages and share location
* Python library: [hangups](https://github.com/tdryer/hangups)


# Hangups
* by Tom Dryer, MIT licensed
* reverse-engineered the Hangouts protocol
* provides a basic command-line chatting client
* supports OAuth2 authentication as "iOS device"
* quite active, already some bots use it
* [https://github.com/tdryer/hangups](https://github.com/tdryer/hangups)


#Event-driven Programming
> In an event-driven program, program flow is determined by external events. It is characterized by an *event loop* and the use of callback to trigger actions when events happen. - **Jessica McKellar, Twisted Network Programming Essentials**

* main loop listens for events and triggers *callback functions* (continuations) when one of these events is detected
* mostly single-threaded but multi-threaded architectures also exist
* can be blocking (synchronous) or non-blocking (asynchronous)

#Asynchronous Programming
> Asynchronous I/O, or non-blocking I/O is a form of input/output processing that permits other processing to continue before the transmission has finished - **Wikipedia**

* a form of cooperative multi-tasking
* concept can be implemented as an event loop

#When to use asynchronous event-driven programming?
* if you have many tasks
* your tasks are largely independent (no much inter-communication)
* some tasks block while waiting for I/O, events...
* your tasks share mutable data (that would need to be synced)

Escpecially *network applications* and *user interfaces* have exactly these properties

![single threaded](gfx/asynchronous_st.png)

![multi-threaded](gfx/asynchronous_mt.png)

![asynchronous](gfx/asynchronous_async.png)

# Examples

**Task**: Fetch some urls and check the elapsed time

1. single-threaded
1. multi-threaded
1. asynchronous

# Single-threaded
&#8203;

In [2]:
import urllib, time, hashlib

hosts = ['http://www.scikit-learn.org', 'http://www.numpy.org', 'http://www.scipy.org', 'http://pandas.pydata.org']

start = time.time()
for host in hosts:
    url = urllib.request.urlopen(host)
    print(url.read().upper()[:20], host)
print("Elapsed time: {}".format(time.time() - start))

b'<HTML>\n<HEAD>\n  <TIT' http://www.scikit-learn.org
b'\n<!DOCTYPE HTML PUBL' http://www.numpy.org
b'<!DOCTYPE HTML>\n\n<HT' http://www.scipy.org
b'<!DOCTYPE HTML PUBLI' http://pandas.pydata.org
Elapsed time: 1.3736803531646729


# Multi-threaded
&#8203;

In [3]:
import threading

def print_page(host):
    url = urllib.request.urlopen(host)
    print(url.read().upper()[:20], host)

# generate jobs
jobs = list()
for host in hosts:
    jobs.append(threading.Thread(target=print_page, args=(host,)))
    
start = time.time()
# start jobs
for job in jobs:
    job.start()
    
# wait for jobs to finish
for job in jobs:
    job.join()
print("Elapsed time: {}".format(time.time() - start))

b'\n<!DOCTYPE HTML PUBL' http://www.numpy.org
b'<HTML>\n<HEAD>\n  <TIT' http://www.scikit-learn.org
b'<!DOCTYPE HTML>\n\n<HT' http://www.scipy.org
b'<!DOCTYPE HTML PUBLI' http://pandas.pydata.org
Elapsed time: 0.5780215263366699


# Asynchronous 
&#8203;

In [36]:
# This code runs only with Python 2
from twisted.internet import defer, task
from twisted.web.client import getPage

start = time.time()

def print_capitalized(txt, host):
    print(txt.upper()[:20], host)
    
def print_elapsed_time(result):
    print("Elapsed time: {}".format(time.time() - start))
    
def main(react, hosts):
    dlist = list()
    for host in hosts:
        d = getPage(host)
        d.addCallback(print_capitalized, host)
        dlist.append(d)
    
    return defer.gatherResults(dlist).addCallback(print_elapsed_time)

task.react(main, (hosts,))

Elapsed time: 0.423393964767

# Some thoughts about callbacks

> It requires super human descipline to write readable code in callbacks and if you don't believe me look at any piece of JavaScript code - **Guido van Rossum**

# Asynchronous 
&#8203;

In [4]:
import asyncio
import aiohttp

start = time.time()

@asyncio.coroutine
def print_page(host):
       response = yield from aiohttp.request('GET', host)
       page = yield from response.read()
       print(page.upper()[:20], host)

tasks = list()
for host in hosts:
  tasks.append(print_page(host))

loop = asyncio.get_event_loop()
loop.run_until_complete(asyncio.wait(tasks))

print("Elapsed time: {}".format(time.time() - start))

b'\n<!DOCTYPE HTML PUBL' http://www.numpy.org
b'<!DOCTYPE HTML>\n\n<HT' http://www.scipy.org
b'<HTML>\n<HEAD>\n  <TIT' http://www.scikit-learn.org
b'<!DOCTYPE HTML PUBLI' http://pandas.pydata.org
Elapsed time: 0.332597017288208


# What should our MedBot do?
![bot](gfx/bot.png)
* notify a friend at 8pm about taking his long-acting insulin using Google Talk/Hangouts
* wait 20min for a reply then ask again, repeat this twice
* when message is retrieved check for "yes" and praise him, do nothing if no "yes"
* after having asked three times in total, i.e. 60min later, send give up message

# State machine

![state_machine](gfx/state_machine.png)

# Implementation

* The implementation uses the concepts introduced above.
* Checkout [https://github.com/blue-yonder/medbot/](https://github.com/blue-yonder/medbot/).
 * **src/medbot_xmpp.py** for an implementation using XMPP
 * **src/medbot_asyncio.py** for an implementation using ASYNCIO
 

# OAuth2
* open standard for authorization
* allows resource owners to authorize third-party access to their resources *without sharing their credentials*
* clients use access-tokens to access the protected resources on a server
* [https://developers.google.com/identity/protocols/OAuth2](https://developers.google.com/identity/protocols/OAuth2)

![webflow by Google, CC 3.0](gfx/webflow.png)
[https://developers.google.com/identity/protocols/OAuth2InstalledApp](https://developers.google.com/identity/protocols/OAuth2InstalledApp)

# How to use it?

1. create a project with the Google Developers Console
1. give some details for the consent screen
1. create OAuth client credentials (id and secret)
1. request token for a scope
1. get the user's consent to get authorization code
1. exchange code for a token


# Google Developers Console
![Google Developers Console](gfx/dev_console_projects.png)
[https://console.developers.google.com/](https://console.developers.google.com/)

# Consent screen
![Consent Screen](gfx/authentication.png)

# Settings for Consent screen
![](gfx/dev_console_consent.png)

# Settings for client credentials
![](gfx/dev_console_credentials.png)

![](gfx/dev_console_create_client.png)

# Client credentials
![](gfx/dev_console_client_credentials.png)

# Request a token for a scope

* just form a GET request and let the user confirm
* use the scope for Google Talk (https://www.googleapis.com/auth/googletalk)
* get the user's consent and retrieve the authentication code

In [15]:
import urllib
params = dict(client_id='10250...com',
              scope='https://www.googleapis.com/auth/googletalk',
              redirect_uri='urn:ietf:wg:oauth:2.0:oob',
              response_type='code')
url = 'https://accounts.google.com/o/oauth2/auth?{}'.format(urllib.parse.urlencode(params))

# Consent screen
![Consent Screen](gfx/authentication.png)

# Authentication Code
![Authentication code](gfx/authentication_code.png)

# Exchange code for token

* use the authentication code to form a POST request


In [16]:
import requests
params = dict(client_id='10250...com',
              client_secret='qlEM...A6GQ',
              code='4/504...PM85',
              grant_type='authorization_code',
              redirect_uri='urn:ietf:wg:oauth:2.0:oob')
res = requests.post('https://www.googleapis.com/oauth2/v3/token', data=params).json()

In [17]:
res = {'expires_in': 3600, 'access_token': 'ya29.sEFNeZZcHIG-sdfelULRscbJmOoZ6F5HELsun_rEs15VASul_2g__2GPM0BH4Vs3Jgq', 'token_type': 'Bearer', 'refresh_token': '1/GM3Bme7d8phaFRlnmKE3Gb_YwWdk9bDsqPI&J-kxlHBIgOrIDttvn6zK6XiATCKT'}


In [18]:
import pprint
pp = pprint.PrettyPrinter()
pp.pprint(res)

{'access_token': 'ya29.sEFNeZZcHIG-sdfelULRscbJmOoZ6F5HELsun_rEs15VASul_2g__2GPM0BH4Vs3Jgq',
 'expires_in': 3600,
 'refresh_token': '1/GM3Bme7d8phaFRlnmKE3Gb_YwWdk9bDsqPI&J-kxlHBIgOrIDttvn6zK6XiATCKT',
 'token_type': 'Bearer'}


# Thanks for your attention
![bot](gfx/bot_waves.png)

# Credits
* Thanks to Tom Dryer for some explanations about hangups
* OAuth2 webflow diagram was created and [shared by Google](https://developers.google.com/readme/policies/) and used according to terms described in the [Creative Commons 3.0 Attribution License](http://creativecommons.org/licenses/by/3.0/). The image was scaled and white changed to alpha.